# Regressão Linear Múltipla

## Carregando o Dataset Boston Houses

1. CRIM: per capita crime rate by town 
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft. 
3. INDUS: proportion of non-residential acres per town 
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) 
5. NOX: nitric oxides concentration (parts per 10 million) 
6. RM: average number of rooms per dwelling 
7. AGE: proportion of owner-occupied units built prior to 1940 
8. DIS: weighted distances to five Boston employment centres 
9. RAD: index of accessibility to radial highways 
10. TAX: full-value property-tax rate per 10,000 
11. PTRATIO: pupil-teacher ratio by town 
12. B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town 
13. LSTAT: % lower status of the population 
14. TARGET: Median value of owner-occupied homes in $1000's

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
%matplotlib inline 

In [ ]:
# Gerando o dataset
boston = load_boston() 
dataset = pd.DataFrame(boston.data, columns = boston.feature_names)
dataset['target'] = boston.target

In [ ]:
dataset.head()

In [ ]:
len(dataset)

## Análise Descritiva

In [ ]:
dataset.describe().T

In [ ]:
dataset.describe()['target'] # variável preditora ou Classe ou Label

In [ ]:
dataset.info()

## Gerando número de observações 

In [ ]:
observations = len(dataset)
observations

In [ ]:
dataset.head()

## Coletando x e y

In [ ]:
dataset.iloc[:,:-1][:3]

In [ ]:
X = dataset.iloc[:,:-1]
y = dataset['target'].values

In [ ]:
X.head()

In [ ]:
y[:5]

### Matriz de Correlação

In [ ]:
# Gerando a matriz
X = dataset.iloc[:,:-1]
matriz_corr = X.corr()
print (matriz_corr)

In [ ]:
# matriz de Correlação com a variável preditora
mt = pd.DataFrame(dataset.values, columns=dataset.columns)
mt_corr = mt.corr()
print (abs(mt_corr['target']).sort_values(ascending=False))

## Visualizando a matriz de correlação (somente os atributos)

In [ ]:
# Criando um Correlation Plot
def visualize_correlation_matrix(data, hurdle = 0.0):
    fig = plt.figure(figsize=(12,9))
    ax = fig.add_subplot(111)
    R = np.corrcoef(data, rowvar = 0)
    R[np.where(np.abs(R) < hurdle)] = 0.0
    heatmap = plt.pcolor(R, cmap = mpl.cm.coolwarm, alpha = 0.8)
    heatmap.axes.set_frame_on(False)
    heatmap.axes.set_yticks(np.arange(R.shape[0]) + 0.5, minor = False)
    heatmap.axes.set_xticks(np.arange(R.shape[1]) + 0.5, minor = False)
    heatmap.axes.set_xticklabels(dataset.columns[:-1], minor = False)
    plt.xticks(rotation=90)
    heatmap.axes.set_yticklabels(dataset.columns[:-1], minor = False)
    plt.tick_params(axis = 'both', which = 'both', bottom = 'off', top = 'off', left = 'off', right = 'off')   
    plt.colorbar()
    plt.show()

In [ ]:
# Visualizando o Plot
visualize_correlation_matrix(X, hurdle = 0.5)

## Feature Scaling

In [ ]:
X.head()

Podemos aplicar Feature Scaling através de Padronização ou Normalização. <br />Normalização aplica escala aos dados com intervalos entre 0 e 1. <br />A Padronização divide a média pelo desvio padrão para obter uma unidade de variância. <br />Vamos usar a Padronização (StandardScaler) pois nesse caso esta técnica ajusta os coeficientes e torna a superfície de erros mais "tratável".

### Aplicando Padronização

In [ ]:
standardization = StandardScaler()
Xst = standardization.fit_transform(X)
original_means = standardization.mean_
originanal_stds = standardization.scale_
print('Dataset Original')
X.head()

In [ ]:
print('Dataset Padronizado')
dstd = pd.DataFrame(Xst, columns=boston.feature_names)
dstd.head()

In [ ]:
X.max()

In [ ]:
dstd.max()

In [ ]:
X.min()

In [ ]:
dstd.min()

In [ ]:
y[:5]

### Desfazendo a Padronização

In [ ]:
dfXst = pd.DataFrame(Xst, columns=boston.feature_names)
dfXst.head()

In [ ]:
Xinv = standardization.inverse_transform(Xst)
dfinverser = pd.DataFrame(Xinv, columns=boston.feature_names)
dfinverser.head()

In [ ]:
dataset.head()

# Criar o modelo

In [ ]:
# Criando um modelo
modelo = linear_model.LinearRegression() #normalize = False, fit_intercept = True)
modelo.fit(Xst,y)

## Mostrar os pesos dos Atributos ( yprev = w1X1 + w2X2 + ... + wnXn )

In [ ]:
for coef, var in sorted(zip(modelo.coef_, dataset.columns[:-1]), reverse = True):
    print ("%6.3f %s" % (coef,var))

### Avaliando o modelo com o R Squared (R²) 

In [ ]:
def r2_est(X,y):
    modelo = linear_model.LinearRegression(normalize = False, fit_intercept = True)
    return r2_score(y, modelo.fit(X,y).predict(X))

In [ ]:
print ('R2: %0.3f' %  r2_est(Xst,y))

### Gera o impacto de cada atributo no R² 

In [ ]:
r2_impact = list()
for j in range(X.shape[1]):
    selection = [i for i in range(X.shape[1]) if i!=j]
    r2_impact.append(((r2_est(X,y) - r2_est(X.values[:,selection],y)), dataset.columns[j]))
    
for imp, varname in sorted(r2_impact, reverse = True):
    print ('%6.3f %s' %  (imp, varname))

In [ ]:
0.056 LSTAT
 0.044 RM
 0.029 DIS
 0.028 PTRATIO
 0.011 NOX
 0.011 RAD
 0.006 B
 0.006 ZN
 0.006 CRIM
 0.006 TAX
 0.005 CHAS
 0.000 INDUS
 0.000 AGE

# Fazer Previsões

In [ ]:
dataset.tail()

In [ ]:
#             CRIM	  ZN  INDUS	  CHAS	     NOX      RM	 AGE	DIS	  RAD	  TAX	 PTRATIO	B	   LSTAT
Xteste = [   30.,   0.0, 11,      0.0,      0.5,     6. ,   80.2,  2.3,  1.0,    273.0,   21.2,      396.1,  7.4
    ]

standardization = StandardScaler()
Xteste_escala = standardization.fit_transform(np.array(Xteste).reshape(1, -1)) 
#m = modelo.fit(X,y)
#Xteste_norm = standardization.fit_transform(X)
#modelo.predict(np.array(Xteste).reshape(1, -1))[0]

modelo.predict(Xteste_escala)[0]

In [ ]:
Xteste = [
    [   0.02,   0.2, 7.01,   0.0,   0.5,  7.1,  45.2,  6.1,  3.0,    222,   15.2,      396.1,   5.4],
    [   0.01,   0.1,11.01,   0.0,   0.6,  6.1,  80.2,  2.5,  1.0,    273,   21.2,      396.9,   12.6],
]


Xteste_escala = standardization.fit_transform(np.array(Xteste)) 

modelo.predict(np.array(Xteste_escala))

# Métricas para Algoritmos de Regressão

### Gerando o dataset

In [ ]:
dataset['y_prev'] = modelo.predict(Xst)
#dataset['Erro'] = dataset['y_prev'] - dataset['target'] 
dataset.head()

In [ ]:
dataset['Erro'] = abs (dataset['y_prev'] - dataset['target'] )
dataset.head()

In [ ]:
dataset.Erro.sum()

## MAE - Mean Absolute Error
É a soma da diferença absoluta entre previsões e valores reais.<br />
Fornece uma ideia de quão erradas estão nossas previsões.<br />
Valor igual a 0 indica que não há erro, sendo a previsão perfeita <br />
(a exemplo do Logloss, a função cross_val_score inverte o valor) 

In [ ]:
from sklearn import model_selection
kfold = 10
resultado = model_selection.cross_val_score(modelo, Xst, y, cv = kfold, scoring = 'neg_mean_absolute_error')

# Print do resultado
print("MAE: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

## MSE - Mean Squared Error
Similar ao MAE, fornece a magnitude do erro do modelo.<br />
Ao extrairmos a raiz quadrada do MSE convertemos as unidades de volta ao original, 
o que pode ser útil para descrição e apresentação.<br />
Isso é chamado RMSE (Root Mean Squared Error)

In [ ]:
# Definindo os valores para o número de folds
num_folds = 10
num_instances = len(X)
seed = 7

# Separando os dados em folds
kfold = model_selection.KFold(num_folds, True, random_state = seed)

resultado = model_selection.cross_val_score(modelo, Xst, y, cv = kfold, scoring = 'neg_mean_squared_error')

# Print do resultado
print("MSE: %.3f (%.3f)" % (resultado.mean(), resultado.std()))

# RMSE (Root Mean Squared Error)

Similar ao MAE, fornece a magnitude do erro do modelo.<br>
Ao extrairmos a raiz quadrada do MSE convertemos as unidades de volta ao original, o que pode ser útil para descrição e apresentação.

In [ ]:
from math import sqrt
print("RMSE: %.3f " % (sqrt(abs(resultado.mean()))))

## R2  
Essa métrica fornece uma indicação do nível de precisão das previsões em relação aos valores observados.<br />
Também chamado de coeficiente de determinação.<br />
Valores entre 0 e 1, sendo 1 o valor ideal.

In [ ]:
resultado = model_selection.cross_val_score(modelo, Xst, y, cv = kfold, scoring = 'r2')

# Print do resultado
print("R^2: %.3f (%.3f)" % (resultado.mean(), resultado.std()))